# Tarea 1b: generar texto mediante una petición que incluya contexto

En este cuaderno, aprenderá a generar una respuesta de correo electrónico a un cliente que no se sintió a gusto con la calidad del servicio al cliente que recibió del ingeniero de soporte. Proporcionará contexto adicional al modelo con el contenido del correo electrónico real que recibió del cliente insatisfecho.

Agregará más complejidad con la ayuda de PromptTemplates para aprovechar el marco de trabajo LangChain para un caso práctico similar. Las PromptTemplates permiten crear shells genéricos que se pueden completar con información más adelante y obtener salidas del modelo en función de distintas situaciones.

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) es un marco de trabajo que permite desarrollar aplicaciones con tecnología de modelos de lenguaje. Los aspectos clave de este marco de trabajo nos permiten ampliar los modelos de lenguaje grandes, ya que es posible unir diversos componentes para crear casos prácticos avanzados.

Debido al contexto adicional de la petición, el contenido que se genera en este cuaderno es de mejor calidad y de mayor relevancia que el creado anteriormente con peticiones sin entrenamiento previo. La petición que se usa en este cuaderno crea una plantilla de peticiones de LangChain personalizada que permite agregar contexto a la solicitud de generación de texto.

#### Situación
Usted es Bob, un gerente de servicio al cliente en AnyCompany, y algunos de sus clientes no se sienten satisfechos con el servicio al cliente, por lo que realizan comentarios negativos en relación con el servicio proporcionado por los ingenieros de soporte. Ahora, le gustaría responder a esos clientes y disculparse con humildad por el deficiente servicio para recuperar su confianza. Necesita contar con la ayuda de un modelo de lenguaje grande (LLM) para generar correos electrónicos masivos que sean amigables con los clientes y estén personalizados en función del sentimiento que experimentaron con los correos electrónicos anteriores.

En esta situación, puede aprovechar la eficacia de PromptTemplates de LangChain para crear un shell genérico para generar respuestas de correo electrónico personalizadas a partir del correo electrónico anterior del cliente. La PromptTemplate incorporará el contenido original del correo electrónico del cliente, lo que permitirá al LLM comprender el contexto y el sentimiento y, luego, generar una respuesta adecuada y personalizada.

## Tarea 1b.1: configuración del entorno

En esta tarea, establecerá el entorno.

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))


bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## Tarea 1b.2: invocar el modelo LLM de Bedrock

En esta tarea, se crea una instancia de la clase Bedrock a partir de los LLM. Se espera un `model_id`, que es el Amazon Resource Name (ARN) del modelo disponible en Amazon Bedrock.

Si lo desea, puede pasar un cliente boto3 creado anteriormente, así como algunos `model_kwargs` que pueden contener parámetros como `temperature`, `top_p`, `max_token_count` o `stop_sequences` (puede obtener más información sobre los parámetros en la consola de Amazon Bedrock).

Consulte la [documentación](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) para conocer los ID de modelo de generación de texto disponibles en Amazon Bedrock.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Nota:** Los distintos modelos admiten diferentes `model_kwargs`.

In [ ]:
# Model configuration
from langchain_aws import ChatBedrock
from langchain_core.output_parsers import StrOutputParser

model_id = "meta.llama3-8b-instruct-v1:0"
model_kwargs =  { 
        "max_gen_len": 512,
        "temperature": 0,
        "top_p": 1,
}

# LangChain class for chat
chat_model = ChatBedrock(
    client=bedrock_client,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

## Tarea 1b.3: crear una plantilla de peticiones personalizada de LangChain

En esta tarea, creará una plantilla para la petición a la que puede pasar diferentes variables de entrada en cada ejecución. Esto es útil cuando debe generar contenido con diferentes variables de entrada que tal vez recopila de una base de datos.

En la tarea anterior, codificamos la petición. Es posible que existan múltiples clientes que envíen comentarios negativos similares, y que usted desee responder con una disculpa a cada uno de sus correos electrónicos, pero también ofrecer respuestas que incluyan elementos personalizados. En la siguiente celda, analizará cómo crear una `PromptTemplate` para lograr este patrón.

In [ ]:
# Create a prompt template that has multiple input variables
from langchain.prompts import PromptTemplate

multi_var_prompt = PromptTemplate(
    input_variables=["customerServiceManager", "customerName", "feedbackFromCustomer"], 
    template="""

Human: Create an apology email from the Service Manager {customerServiceManager} at AnyCompany to {customerName} in response to the following feedback that was received from the customer: 
<customer_feedback>
{feedbackFromCustomer}
</customer_feedback>

Assistant:"""
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(customerServiceManager="Bob Smith", 
                                 customerName="John Doe", 
                                 feedbackFromCustomer="""Hello Bob,
     I am very disappointed with the recent experience I had when I called your customer support.
     I was expecting an immediate call back but it took three days for us to get a call back.
     The first suggestion to fix the problem was incorrect. Ultimately the problem was fixed after three days.
     We are very unhappy with the response provided and may consider taking our business elsewhere.
     """
     )


In [ ]:
# get number of tokens
num_tokens = chat_model.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Nota:** Puede ignorar las advertencias sin problema y pasar a la siguiente celda.

In [ ]:
#invoke
response = chat_model.invoke(prompt)

In [ ]:
# Configure a Chain to parse output
chain = StrOutputParser()
formatted_response=chain.invoke(response)
print(formatted_response)

Aprendió correctamente que invocar el LLM sin contexto puede no generar los resultados deseados. Al agregar contexto y utilizar la plantilla de peticiones para restringir la salida del LLM, pudo obtener la salida deseada correctamente.

### Pruébelo usted mismo
- Cambie las peticiones para que se adapten al caso práctico específico y evalúe la salida de los diferentes modelos.
- Juegue con la longitud del token para comprender la latencia y la capacidad de respuesta del servicio.
- Aplique diferentes principios de ingeniería de peticiones para obtener mejores salidas.

### Limpieza

Ha completado este cuaderno. Para ir a la siguiente parte del laboratorio, complete estos pasos:

- Cierre este archivo de cuaderno.
- Regrese a la sesión de laboratorio y continúe con la **Tarea 2**.